# Merkle Tree and Lamport signature

This notebook explains how to use our Python 3 implementation of Merkle Tree and Lamport signature, using the SHA-256 hash function.


FIrst, import Merkle tree and Lamport signature classes:

In [1]:
from src.lamport import LamportSignature
from src.merkle_tree import MerkleTree

The Merkle signature scheme can be used to sign a limited number of messages N with one public key pub. The number of possible messages must be a power of two, so we denote the possible number of messages as N = 2**n.

In [2]:
N = 4  # Number of messages to sign

Generate 'N' (private, public) key pairs (Xi, Yi) from the Lamport signature scheme:

In [3]:
key_pairs = [LamportSignature() for _ in range(N)]  # Private key of the Merkle signature scheme

The leaves of the tree are the hashed values of the public keys Y0, ..., YN:

In [4]:
mk = MerkleTree(n_leaves=N)
for i in range(N):
    mk.add_node(key_pairs[i].get_key('public', concatenate=True), (0, i), hashed=False)

Next, we build Merkle tree:

In [5]:
mk.generate_tree()
pub = mk.get_root()  # Public key of the Merkle signature scheme

Merkle signature generation using a chosen pair of keys (Xi, Yi) from the Lamport signature scheme:

In [6]:
pair = 3  # (Xi, Yi) pair number
sig = []  # Merkle signature
M = "test"  # message to sign
sig_prime = key_pairs[pair].sign(M)  # Lamport signature
sig.append(sig_prime)  # Add sig_prime to the signature 'sig'.
sig.append(key_pairs[pair].get_key('public', concatenate=True))  # Add Yi to the signature 'sig'.
sig.append(mk.get_authentification_path_hashes(pair))  # Add auth(0), ..., auth(n-1) to the signature 'sig'.

Merkle signature verification:
Receiver knows the public key 'pub' (tree root), the message 'M' and the Merkle signature 'sig'.

In [7]:
pub_receiver = pub
M_receiver = M
sig_receiver = sig

First, the receiver verifies the one time signature 'sig_prime' of the message 'M' using the Lamport key 'Yi':

In [8]:
print("Check one time signature of the received message: " + M_receiver)
result = LamportSignature.verify(M_receiver, sig_receiver[0], LamportSignature.decatenate_key(sig_receiver[1]))
print("One-time signature is: " + str(result))

Check one time signature of the received message: test
One-time signature is: True


If 'sig_prime' is a valid signature of 'M', the receiver generates the Merkle tree and compare the root to the signature:

In [9]:
if result:
        mk_receiver = MerkleTree(n_leaves=N)
        mk_receiver.add_node(sig_receiver[1], (0, pair), hashed=False)
        mk_receiver.generate_tree()

        # The nodes of the authentification path are computed.
        for i, (level, index) in enumerate(mk_receiver.get_authentification_path(pair)):
            mk_receiver.add_node(sig_receiver[2][i], (level, index), hashed=True)
        mk_receiver.generate_tree()

        # If the tree's root equals the public key 'pub' the signature is valid.
        result = mk_receiver.get_root() == pub_receiver
        print("Merkle signature is: " + str(result))

Merkle signature is: True


Because the Lamport signature scheme is a one-time signature scheme you must not use the same (public, private) keys several times:

In [11]:
pair = 3  # Re-use the same (Xi, Yi) pair number
M = "test the same key pair again"  # message to sign
sig_prime = key_pairs[pair].sign(M)  # Lamport signature

ValueError: Private and public keys already used!